# What is Selenium?

## 1

What environment do we use?

See [What is Docker?](What%20is%20Docker.ipynb)

How to set up selenium server?


In [7]:
var importer = require('../Core');
var path = require('path');
var fs = require('fs');
var execCmd = importer.import('spawn child process');
var importer = require('../Core');
var seleniumDocker = importer.import('selenium docker');

var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '', 'Downloads');
var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '', '.defaultProfile');
var DOCKERFILE = path.resolve(path.join(__dirname, 'Dockerfile'));

function getSeleniumServer(name = 'act-selenium') {
    try {
        fs.mkdirSync(DOWNLOAD_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.mkdirSync(PROFILE_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));
    } catch (err) {
        if (err.code != 'ENOENT') {
            throw err;
        }
    }
    seleniumDocker(DOCKERFILE);
    return execCmd('docker ps -a')
        .then(r => {
            if (r[0].indexOf(name) > -1) {
                return execCmd('docker stop ' + name)
                    .then(r => new Promise(resolve =>
                        setTimeout(() => resolve(r), 1000)))
                    .then(() => execCmd('docker rm ' + name));
            }
        })
        .then(() => new Promise(resolve =>
            setTimeout(() => resolve(), 1000)))
        .then(() => {
            var build = 'docker build -t ' + name + ' "'
                + path.dirname(DOCKERFILE) + '"\n'
                + 'docker run --shm-size=3g -d '
                + '--name ' + name + ' '
                + '-p 8888:8888 '
                + '-p 6080:6080 '
                + '-p 5900:5900 '
                + '-p 4444:4444 '
                + '-p 4200:4200 '
                + '-p 3000:3000 '
                // TODO: add profile dir back in when permissions works on windows
                + '-v "' + DOWNLOAD_DIR + '":/data/downloads '
                + name + '\n';
            return execCmd(build)
        })
        .then(r => new Promise(resolve => setTimeout(() => resolve(r), 6000)))
};
module.exports = getSeleniumServer;

if(typeof $$ !== 'undefined') {
    $$.async();
    getSeleniumServer()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e));
}


[Function: getSeleniumServer]

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser


In [1]:
function urlEncode(obj) {
    return Object.keys(obj).map(k => encodeURIComponent(k) + '=' + encodeURIComponent(obj[k])).join('&');
}

var vncIframe = (options = {
    password: 'secret',
    host: 'localhost',
    port: 6080,
    autoconnect: true,
    resize: 'downscale',
    view_only: true,
    reconnect: true
}) => `
<div style="display:block; width:100%; padding-bottom:69%;position:relative;">
<iframe id="vnc"
style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" 
src="//local:6080/vnc.html?${urlEncode(options)}"></iframe></div>
`;
module.exports = vncIframe;

if(typeof $$ !== 'undefined') {
    $$.mime({'text/html': vncIframe()});
}


[Function: vncIframe]

## 3

How to run a selenium cell on the Docker machine?



In [11]:
var importer = require('../Core');

function runSeleniumCell(search, newWindow = true) {
    var createWebdriverClient = importer.import('webdriver client');
    var client = createWebdriverClient();
    var ctx = {client};
    var {
        getCredentials,
        getAllXPath,
        getAllUntil,
        resizeWindow,
        onlyOneWindow,
        getAllSessionUrls,
        clickSpa,
    } = importer.import([
        'decrypt password',
        'all elements xpath',
        'get all elements until',
        'resize selenium window',
        'only one window',
        'get all session urls',
        'click spa link',
    ], ctx);
    var formUtilities = importer.import('form utilities', ctx);
    var isError = false;
    return client
        .scanning(true)
        .getSessions()
        .then(() => (newWindow
                     ? client
                     .then(() => onlyOneWindow())
                     .then(() => getAllSessionUrls())
                     : []))
        .catch(e => {
            console.log(e);
            isError = e;
            throw new Error('there is an error with the client ' + e);
        })
        .scanning(false)
        .then(() => {
            Object.assign(ctx, {
                client,
                browser: client,
                getCredentials,
                getAllXPath,
                getAllUntil,
                resizeWindow,
                clickSpa,
            }, formUtilities);
            if(!search) {
                return ctx;
            }
            return importer.import(search, ctx)
        })
};
module.exports = runSeleniumCell;

if(typeof $$ !== 'undefined') {
    console.log('this should not be hit!');
    $$.async()
    runSeleniumCell('test docker selenium')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e));
}


[Function: runSeleniumCell]

Test docker selenium?

In [ ]:
function testLive() {
    return client
        .url('https://purchasesprint.actops.com')
        .click('[href*="/auth"], [routerlink*="/auth"]');
}
module.exports = testLive();




TODO:

Auto fill fields in a live browser using the 

Transfer login state to current browser?

